In [24]:
import sqlite3
import os
import pandas as pd

In [28]:
pip install ipython-sql

     |████████████████████████████████| 42 kB 2.3 MB/s eta 0:00:011
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=5e30a945bff9c6a6965324464f379ecba9ed974c7c91afbc0f8b08da259b1f2d
  Stored in directory: /Users/nadiawood/Library/Caches/pip/wheels/75/f7/28/77a076f1fa8cbeda61aca712815d04d7a32435f04a26a2dd7b
Successfully built prettytable
Note: you may need to restart the kernel to use updated packages.


In [30]:
%reload_ext sql

In [31]:
%%bash
pwd
ls ../2CleanedData

/Users/nadiawood/Documents/GitHub/cs513-data-cleaning/SQL
OpenRefine
farmeresmarkets_payments.csv
farmersMarket_location.csv
farmersmarket_impute_zip.ipynb
farmersmarkets_output.csv
farmersmarkets_products.csv


In [32]:
df = pd.read_csv("../2CleanedData/farmersMarket_location.csv")

In [33]:
df.shape

(8675, 10)

## Commands for sqllite

In [21]:
%%bash
sqlite3 ../farmersmarkets.db

.header on
.mode csv


CREATE TABLE location(
	"FMID"			INT PRIMARY KEY NOT NULL,
	"MarketName" 	TEXT,
	"street"		TEXT,
	"City"			TEXT,
	"County"		TEXT,
	"State"			TEXT,
	"zip"			TEXT,
	"latitude"		REAL,
	"longitude"		REAL,
	"updateTime"	DATETIME
);
.import /Users/nadiawood/Documents/GitHub/cs513-data-cleaning/2CleanedData/farmersMarket_location.csv location

CREATE TABLE payments(
	"FMID"			INT PRIMARY KEY NOT NULL,
	"Credit"		CHAR(1),
	"WIC"			CHAR(1),
	"WICcash"		CHAR(1),
	"SFMNP"			CHAR(1),
	"SNAP"			CHAR(1)
);
.import /Users/nadiawood/Documents/GitHub/cs513-data-cleaning/2CleanedData/farmeresmarkets_payments.csv payments

CREATE TABLE products(
	"FMID"			INT PRIMARY KEY NOT NULL,
	"Organic"		CHAR(1),
	"Bakedgoods"	CHAR(1),
	"Cheese"		CHAR(1),
	"Crafts"		CHAR(1),
	"Flowers"		CHAR(1),
	"Eggs"			CHAR(1),
	"Seafood"		CHAR(1),
	"Herbs"			CHAR(1),
	"Vegetables"	CHAR(1),
	"Honey"			CHAR(1),
	"Jams"			CHAR(1),
	"Maple"			CHAR(1),
	"Meat"			CHAR(1),
	"Nursery"		CHAR(1),
	"Nuts"			CHAR(1),
	"Plants"		CHAR(1),
	"Poultry"		CHAR(1),
	"Prepared"		CHAR(1),
	"Soap"			CHAR(1),
	"Trees"			CHAR(1),
	"Wine"			CHAR(1),
	"Coffee"		CHAR(1),
	"Beans"			CHAR(1),
	"Fruits"		CHAR(1),
	"Grains"		CHAR(1),
	"Juices"		CHAR(1),
	"Mushrooms"		CHAR(1),
	"PetFood"		CHAR(1),
	"Tofu"			CHAR(1),
	"WildHarvested"	CHAR(1)
);
.import /Users/nadiawood/Documents/GitHub/cs513-data-cleaning/2CleanedData/farmersmarkets_products.csv products

.output ./farmersmarkets.sql
.dump

## Connect to database

In [22]:
# Connect to database. Following command will connect you to the database.
# Any query that you will run after this cell will be run on the `surverys` database.
# You can always change the path in database url and connect to some other sqlite db of your choice.
survey_db_url = 'sqlite:///' + os.path.expanduser('./farmersmarkets.db')
%sql $survey_db_url

UsageError: Line magic function `%sql` not found.


In [23]:
%%sql
SELECT * FROM location LIMIT 5;

UsageError: Cell magic `%%sql` not found.


## Integrity Constraints

### 1. Check that FMID is an appropriate primary key: non-null and unique
Looks good.

In [9]:
%%sql
SELECT * FROM location where FMID IS NULL OR FMID = '';
SELECT COUNT(distinct FMID) from location;

 * sqlite:///./farmersmarkets.db
Done.
Done.


COUNT(distinct FMID)
8687


### 2. Ensure that data for my use case is non-null. Turns out that we have 29 rows where latitude and longitude information are missing.
* latitude and longitude
* state
* credit

In [10]:
%%sql
SELECT *
FROM location loc
WHERE loc.latitude IS NULL 
        OR loc.latitude = ''
        OR loc.longitude IS NULL
        OR loc.longitude = ''
        LIMIT 5;

 * sqlite:///./farmersmarkets.db
Done.


FMID,MarketName,street,City,County,State,zip,latitude,longitude,updateTime
2000001,Center for Design Practice - Mobile Farmers Market,,,,MARYLAND,,,,2013-01-01T05:00:00Z
1011689,Charlotte Regional Farmers Market,1801 YORKMONT ROAD,CHARLOTTE,MECKLENBURG,NORTH CAROLINA,28217,,,2015-11-09T01:23:36Z
2000002,Dig It!,,,,PENNSYLVANIA,,,,2013-01-01T05:00:00Z
1002854,East Goshen Farmers Market,EAST GOSHEN PARK - PAOLI PIKE EAST OF ROUTE 352,WEST CHESTER,CHESTER,PENNSYLVANIA,19380,,,2014-06-05T03:45:15Z
2000004,Farm a la Carte,,,,GEORGIA,,,,2013-01-01T05:00:00Z


In [11]:
%%sql
SELECT COUNT(*)
FROM location loc
WHERE loc.latitude IS NULL 
        OR loc.latitude = ''
        OR loc.longitude IS NULL
        OR loc.longitude = '';

 * sqlite:///./farmersmarkets.db
Done.


COUNT(*)
29


In [12]:
%%sql
SELECT *
FROM location loc
WHERE loc.State IS NULL OR loc.State = '';

 * sqlite:///./farmersmarkets.db
Done.


FMID,MarketName,street,City,County,State,zip,latitude,longitude,updateTime


In [13]:
%%sql
SELECT *
FROM Payments pay
WHERE pay.Credit IS NULL OR pay.Credit = '';

 * sqlite:///./farmersmarkets.db
Done.


FMID,Credit,WIC,WICcash,SFMNP,SNAP


### 3. Check that latitude must be in [0,90] and longitude should be [-180, 180]
Looks good.

In [14]:
%%sql
SELECT loc.FMID,
       loc.MarketName, 
       loc.latitude, 
       loc.longitude 
FROM location loc
WHERE  ( Cast(loc.latitude AS FLOAT) > 180 
          OR Cast(loc.latitude AS FLOAT) <- 180 
          OR Cast(loc.longitude AS FLOAT) < 0 
          OR Cast(loc.longitude AS FLOAT) > 90 ) ; 

 * sqlite:///./farmersmarkets.db
Done.


FMID,MarketName,latitude,longitude


### 4. Every FMID has single address (street, City, County, State, zip) if it exists
Looks good.

In [15]:
%%sql
SELECT
    FMID, street, City, County, State, zip, count(*)
FROM
    location
GROUP BY
    FMID, street, City, County, State, zip
HAVING 
    COUNT(*) > 1

 * sqlite:///./farmersmarkets.db
Done.


FMID,street,City,County,State,zip,count(*)
